In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)

# Parques Urbanos e Lineares com buff de 1km

Para saber quantas pessoas tem no raio de 1km dos parques lineares, podemos determinar esse raio a partir do método buffer e, posteriormente interseccionar os setores censitários (com as informações da população) pela geometria desse buffer.

Neste notebook estaremos apenas realizando o buffer.

In [3]:
data_path=get_data_diretorio()
assets_path= os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook é dependente do asset resultante do notebook '../load_data/malha_parques_lineares_urbano'

In [4]:
gdf_parq_urb_line = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_urbanos_e_lineares.parquet'
    )
)

In [5]:
gdf_parq_urb_line.columns

Index(['cd_parq_mun', 'nm_parque', 'tx_tipo_ca', 'tx_tipo_si', 'cd_tipo_ca',
       'cd_tipo_si', 'geometry'],
      dtype='object')

# Buff

O buff do geopandas é feito a partir de um raio, usando como método a "Soma de Minkowski", ou dilatação morfológica: A+B = {a+b | a E A, b E B}

Como estamos no crs EPSG=31983, podemos simplesmente passar 1000m de raio na distância

In [6]:
buffed= gdf_parq_urb_line[['geometry']].copy()
buffed['geometry'] = buffed['geometry'].buffer(1000)

In [7]:
buffed.sample(2)

,geometry
194,"POLYGON ((321568.265 7401052.43, 321557.726 74..."
142,"POLYGON ((322108.533 7407100.32, 322114.952 74..."


In [8]:
m = buffed.explore()

gdf_parq_urb_line.explore(m=m, color='pink')

# Evitar duplicatas

Como podemos ver, várias polígonos, após o buff, ficam um por cima do outro. Para evitar que isto aconteça usaremos a mesma metodologia usada no notebook '../../area_verde_publica/treat_data/areas_verdes_publicas':

Primeiro usamos a função 'dissolve', que vai transformar todos os polígonos em um grande multipolígono. Então, usando o explode, separamos em polígonos.

In [9]:
buffed=buffed.dissolve()

In [10]:
buffed=buffed.explode(ignore_index=True)

## Salvar GDF

In [11]:
save_parquet_excel(
    buffed,
    'area_1km_parques_urbanos_lineares',
    assets_path,
    'parques_lineares_e_urbanos'
)

